In [1]:
%load_ext autoreload
%autoreload 2
import mycode.vap as vap

In [2]:
# The goal is to experiment how fast we can get to run integration tests with Vespa.
# Starting a vanilla Vespa docker takes some time, deploying an app the first time is also not instant.
# 1. Try to roll back in active version number. Maybe we can freeze some known good session-id and when test is being run we can always roll back to this version.
# 2. Try saving a container with a known good version as an image. Stop the container, and use a snapshot to run some tests with probably measuring the time.

In [3]:
!docker run --rm --detach --name vespa-vanilla --hostname vespa-vanilla --publish 0.0.0.0:8080:8080 --publish 0.0.0.0:19092:19092 --publish 0.0.0.0:19050:19050 --publish 0.0.0.0:19071:19071 vespaengine/vespa:8.600.35

b00c905bd3ec973c2696acd290d0f5e8906291df8ea382981dde88ae84e1b002


In [44]:
# Now let's prepare another application package with the new search chain
from vespa.package import ServicesConfiguration, ApplicationPackage, Schema, Document, Field, RankProfile, Function, \
    FirstPhaseRanking
from vespa.configuration.services import services, container, document_api, search, \
    document_processing, content, documents, document, redundancy

doc_schema = Schema(
    name="doc",
    document=Document(
        fields=[Field(name="id", type="int", indexing=["attribute"])]
    ),
    rank_profiles=[
        RankProfile(
            name='demo',
            inherits='unranked',
            first_phase=
            FirstPhaseRanking(
                expression="attribute(id) * 2",
                keep_rank_count=50,
            )
            ,
            match_features=['attribute(id)'],
        )
    ]
)

application_package = ApplicationPackage(
    name='vanilla',
    schema=[doc_schema],
    services_config=ServicesConfiguration(
        application_name="vanilla",
        services_config=services(
            container(
                document_api(),
                document_processing(),
                search(),
                id="vanilla_container",
                version="1.0"
            ),
            content(
                redundancy(1),
                documents(
                    document(type="doc", mode="index"),
                ),
                id="vanilla_content",
                version="1.0"
            )
        )
    )
)

In [46]:
from pathlib import Path
tmp_vap_folder = '_tmp'
application_package.to_files(Path(tmp_vap_folder))
!vespa deploy _tmp/ -t local

Uploading application package... done;1m⡿
Success: Deployed '_tmp/' with session ID 13


In [14]:
from vespa.application import Vespa
client = Vespa(url="http://localhost", port=8080)

In [15]:
# Create and feed 1 dummy doc
docs = [
    {
        'id': f'{1}',
        'fields': {
            'id': 1,
        }
    }
]

client.feed_iterable(docs, schema="doc", namespace="doc", callback=vap.feed_callback)

In [49]:
client.query(body={
    'yql': 'select * from sources * where true',
    'ranking.profile': 'demo',
}).json

{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 1},
  'coverage': {'coverage': 100,
   'documents': 1,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'children': [{'id': 'id:doc:doc::1',
    'relevance': 2.0,
    'source': 'vanilla_content',
    'fields': {'matchfeatures': {'attribute(id)': 1.0},
     'sddocname': 'doc',
     'documentid': 'id:doc:doc::1'}}]}}

In [ ]:
#12 session

In [48]:
import requests as r
resp = r.put(f'http://localhost:19071/application/v2/tenant/default/session/12/active')
resp

<Response [409]>

In [ ]:
docker commit b00c905bd3ec vespa-vanilla:1
docker run --rm --detach --name vespa-vanilla-1 --hostname vespa-vanilla-1 --publish 0.0.0.0:8080:8080 --publish 0.0.0.0:19092:19092 --publish 0.0.0.0:19050:19050 --publish 0.0.0.0:19071:19071 vespa-vanilla:1